In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
import time
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

warnings.filterwarnings('ignore')

In [2]:
train  = pd.read_csv('competition_data/train.csv')
test  = pd.read_csv('competition_data/test.csv')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 100)

In [3]:
train

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,country,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,1.0,5.0,1.0,5.0,1.0,1.0,USA,3,553,6,4.0,3.0,5.0,1.0,3.0,5.0,5.0,3.0,5.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,2.0,1,3.0,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,1.0,4.0,5.0,3.0,1.0,2.0,4.0,5.0,1.0,3.0,1.0,1.0,5.0,3.0,2.0,5.0,USA,5,85,120,4.0,2.0,3.0,5.0,3.0,2.0,5.0,1.0,2.0,2.0,1,1,1,1,1,0,1,0,0,1,0,0,1,1,1,1,4.0,2,2.0,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,4.0,2.0,5.0,5.0,5.0,1.0,3.0,5.0,3.0,5.0,2.0,2.0,1.0,2.0,4.0,2.0,5.0,NLD,9,108,100,1.0,2.0,3.0,1.0,5.0,5.0,3.0,4.0,5.0,2.0,1,1,0,1,1,0,1,1,0,1,0,0,1,1,1,1,2.0,1,1.0,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,4.0,5.0,2.0,2.0,4.0,4.0,2.0,4.0,5.0,4.0,3.0,3.0,4.0,3.0,4.0,4.0,2.0,USA,2,121,139,3.0,3.0,3.0,4.0,5.0,3.0,4.0,4.0,3.0,3.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,1.0,3,1.0,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,4.0,4.0,3.0,5.0,5.0,2.0,4.0,1.0,4.0,2.0,4.0,2.0,3.0,4.0,4.0,4.0,ITA,3,640,216,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,2.0,1,1,0,1,1,0,0,1,0,1,0,0,0,1,0,1,1.0,2,2.0,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2.0,5.0,4.0,3.0,3.0,4.0,4.0,4.0,3.0,4.0,1.0,4.0,4.0,3.0,4.0,4.0,2.0,5.0,2.0,4.0,1.0,2.0,5.0,4.0,2.0,4.0,USA,12,103,161,2.0,2.0,4.0,3.0,3.0,5.0,3.0,3.0,3.0,3.0,1,1,1,1,1,0,0,0,0,1,0,1,1,1,1,1,2.0,2,2.0,1.0,17,1.0,1.0,3.0,2.0,1.0,3.0,2.0,0
14996,14996,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,1.0,4.0,4.0,4.0,2.0,5.0,5.0,4.0,1.0,5.0,3.0,4.0,4.0,5.0,4.0,5.0,USA,31,106,179,3.0,2.0,4.0,5.0,4.0,3.0,4.0,1.0,2.0,2.0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,1,1,4.0,1,2.0,2.0,45,1.0,3.0,1.0,1.0,2.0,3.0,2.0,1
14997,14997,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0,5.0,3.0,4.0,USA,17,103,168,1.0,3.0,2.0,5.0,1.0,5.0,3.0,3.0,1.0,1.0,1,1,0,1,1,0,0,0,0,1,0,0,1,1,1,1,2.0,2,2.0,1.0,20,1.0,1.0,2.0,1.0,1.0,3.0,1.0,1
14998,14998,5.0,5.0,4.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,3.0,5.0,4.0,4.0,1.0,5.0,4.0,5.0,5.0,2.0,5.0,3.0,5.0,3.0,3.0,5.0,USA,14,68,109,1.0,1.0,3.0,5.0,4.0,5.0,5.0,4.0,2.0,1.0,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,3.0,2,2.0,1.0,29,1.0,12.0,4.0,2.0,2.0,2.0,1.0,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 70 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         15000 non-null  int64  
 1   Q1            14959 non-null  float64
 2   Q2            14931 non-null  float64
 3   Q3            14950 non-null  float64
 4   Q4            14929 non-null  float64
 5   Q5            14962 non-null  float64
 6   Q6            14952 non-null  float64
 7   Q7            14924 non-null  float64
 8   Q8            14952 non-null  float64
 9   Q9            14944 non-null  float64
 10  Q10           14928 non-null  float64
 11  Q11           14941 non-null  float64
 12  Q12           14933 non-null  float64
 13  Q13           14960 non-null  float64
 14  Q14           14964 non-null  float64
 15  Q15           14955 non-null  float64
 16  Q16           14967 non-null  float64
 17  Q17           14963 non-null  float64
 18  Q18           14937 non-nu

In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35452 entries, 0 to 35451
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         35452 non-null  int64  
 1   Q1            35333 non-null  float64
 2   Q2            35305 non-null  float64
 3   Q3            35322 non-null  float64
 4   Q4            35327 non-null  float64
 5   Q5            35362 non-null  float64
 6   Q6            35320 non-null  float64
 7   Q7            35310 non-null  float64
 8   Q8            35344 non-null  float64
 9   Q9            35348 non-null  float64
 10  Q10           35232 non-null  float64
 11  Q11           35333 non-null  float64
 12  Q12           35303 non-null  float64
 13  Q13           35356 non-null  float64
 14  Q14           35350 non-null  float64
 15  Q15           35345 non-null  float64
 16  Q16           35367 non-null  float64
 17  Q17           35373 non-null  float64
 18  Q18           35305 non-nu

In [6]:
train = train.drop(['index', 'country'],axis = 1)
test =test.drop(['index', 'country'],axis = 1)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 68 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            14959 non-null  float64
 1   Q2            14931 non-null  float64
 2   Q3            14950 non-null  float64
 3   Q4            14929 non-null  float64
 4   Q5            14962 non-null  float64
 5   Q6            14952 non-null  float64
 6   Q7            14924 non-null  float64
 7   Q8            14952 non-null  float64
 8   Q9            14944 non-null  float64
 9   Q10           14928 non-null  float64
 10  Q11           14941 non-null  float64
 11  Q12           14933 non-null  float64
 12  Q13           14960 non-null  float64
 13  Q14           14964 non-null  float64
 14  Q15           14955 non-null  float64
 15  Q16           14967 non-null  float64
 16  Q17           14963 non-null  float64
 17  Q18           14937 non-null  float64
 18  Q19           14947 non-nu

In [8]:
train.isna().sum()

Q1               41
Q2               69
Q3               50
Q4               71
Q5               38
Q6               48
Q7               76
Q8               48
Q9               56
Q10              72
Q11              59
Q12              67
Q13              40
Q14              36
Q15              45
Q16              33
Q17              37
Q18              63
Q19              53
Q20              45
Q21              39
Q22              38
Q23              50
Q24              61
Q25              44
Q26              68
introelapse       0
testelapse        0
surveyelapse      0
TIPI1            53
TIPI2            66
TIPI3            79
TIPI4            64
TIPI5            70
TIPI6            62
TIPI7            64
TIPI8            65
TIPI9            64
TIPI10           80
VCL1              0
VCL2              0
VCL3              0
VCL4              0
VCL5              0
VCL6              0
VCL7              0
VCL8              0
VCL9              0
VCL10             0
VCL11             0


In [9]:
#train을 target과 feature로 나눠줍니다.
#train_ = train.dropna(axis=0)
train_= IterativeImputer().fit_transform(train.iloc[:, :-1])
train_= pd.DataFrame(train_)
test= IterativeImputer().fit_transform(test)
test= pd.DataFrame(test)
#train_ = train

In [10]:
train_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 67 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       15000 non-null  float64
 1   1       15000 non-null  float64
 2   2       15000 non-null  float64
 3   3       15000 non-null  float64
 4   4       15000 non-null  float64
 5   5       15000 non-null  float64
 6   6       15000 non-null  float64
 7   7       15000 non-null  float64
 8   8       15000 non-null  float64
 9   9       15000 non-null  float64
 10  10      15000 non-null  float64
 11  11      15000 non-null  float64
 12  12      15000 non-null  float64
 13  13      15000 non-null  float64
 14  14      15000 non-null  float64
 15  15      15000 non-null  float64
 16  16      15000 non-null  float64
 17  17      15000 non-null  float64
 18  18      15000 non-null  float64
 19  19      15000 non-null  float64
 20  20      15000 non-null  float64
 21  21      15000 non-null  float64
 22

In [11]:
norm=Normalizer()
train_S = norm.fit_transform(train_)
train_S = pd.DataFrame(train_S)
test=norm.transform(test) 
test = pd.DataFrame(test)

In [12]:
train_S.columns=['Q1','Q2','Q3','Q4','Q5','Q6','Q7','Q8'
               ,'Q9','Q10','Q11','Q12','Q13','Q14','Q15','Q16'
               ,'Q17','Q18','Q19','Q20','Q21','Q22','Q23','Q24'
               ,'Q25','Q26','introelapse','testelapse','surveyelapse'
                ,'TIPI1','TIPI2','TIPI3'
               ,'TIPI4','TIPI5','TIPI6','TIPI7','TIPI8','TIPI9','TIPI10'
                ,'VCL1','VCL2','VCL3','VCL4','VCL5','VCL6','VCL7'
               ,'VCL8','VCL9','VCL10','VCL11','VCL12','VCL13','VCL14'
               ,'VCL15','VCL16','education','urban','gender','engnat','age'
               ,'hand','religion','orientation','voted','married','familysize'
                ,'ASD']


In [13]:
train_S

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,introelapse,testelapse,surveyelapse,TIPI1,TIPI2,TIPI3,TIPI4,TIPI5,TIPI6,TIPI7,TIPI8,TIPI9,TIPI10,VCL1,VCL2,VCL3,VCL4,VCL5,VCL6,VCL7,VCL8,VCL9,VCL10,VCL11,VCL12,VCL13,VCL14,VCL15,VCL16,education,urban,gender,engnat,age,hand,religion,orientation,voted,married,familysize,ASD
0,0.001805,0.009023,0.009023,0.009023,0.001805,0.007219,0.009023,0.009023,0.001805,0.005414,0.009023,0.009023,0.009023,0.009023,0.009023,0.009023,0.009023,0.001805,0.009023,0.009023,0.001805,0.009023,0.001805,0.009023,0.001805,0.001805,0.005414,0.997974,0.010828,0.007219,0.005414,0.009023,0.001805,0.005414,0.009023,0.009023,0.005414,0.009023,0.005414,0.001805,0.001805,0.000000,0.001805,0.001805,0.0,0.000000,0.000000,0.000000,0.001805,0.000000,0.000000,0.000000,0.001805,0.001805,0.001805,0.003609,0.001805,0.005414,0.001805,0.036093,0.003609,0.021656,0.007219,0.003609,0.001805,0.007219,0.003609
1,0.025531,0.025531,0.025531,0.025531,0.025531,0.031914,0.025531,0.025531,0.019148,0.019148,0.006383,0.025531,0.031914,0.019148,0.006383,0.012766,0.025531,0.031914,0.006383,0.019148,0.006383,0.006383,0.031914,0.019148,0.012766,0.031914,0.031914,0.542536,0.765933,0.025531,0.012766,0.019148,0.031914,0.019148,0.012766,0.031914,0.006383,0.012766,0.012766,0.006383,0.006383,0.006383,0.006383,0.006383,0.0,0.006383,0.000000,0.000000,0.006383,0.000000,0.000000,0.006383,0.006383,0.006383,0.006383,0.025531,0.012766,0.012766,0.006383,0.312756,0.006383,0.012766,0.006383,0.006383,0.012766,0.025531,0.012766
2,0.025724,0.032155,0.032155,0.025724,0.019293,0.032155,0.032155,0.032155,0.025724,0.025724,0.012862,0.032155,0.032155,0.032155,0.006431,0.019293,0.032155,0.019293,0.032155,0.012862,0.012862,0.006431,0.012862,0.025724,0.012862,0.032155,0.057878,0.694537,0.643090,0.006431,0.012862,0.019293,0.006431,0.032155,0.032155,0.019293,0.025724,0.032155,0.012862,0.006431,0.006431,0.000000,0.006431,0.006431,0.0,0.006431,0.006431,0.000000,0.006431,0.000000,0.000000,0.006431,0.006431,0.006431,0.006431,0.012862,0.006431,0.006431,0.012862,0.276529,0.006431,0.012862,0.012862,0.012862,0.019293,0.025724,0.012862
3,0.021454,0.021454,0.021454,0.010727,0.021454,0.016090,0.016090,0.026817,0.016090,0.021454,0.026817,0.010727,0.010727,0.021454,0.021454,0.010727,0.021454,0.026817,0.021454,0.016090,0.016090,0.021454,0.016090,0.021454,0.021454,0.010727,0.010727,0.648973,0.745515,0.016090,0.016090,0.016090,0.021454,0.026817,0.016090,0.021454,0.021454,0.016090,0.016090,0.005363,0.005363,0.000000,0.005363,0.005363,0.0,0.000000,0.000000,0.000000,0.005363,0.000000,0.000000,0.005363,0.005363,0.005363,0.005363,0.005363,0.016090,0.005363,0.005363,0.091178,0.010727,0.005363,0.005363,0.010727,0.005363,0.010727,0.010727
4,0.005916,0.005916,0.005916,0.005916,0.004437,0.004437,0.005916,0.002958,0.004437,0.005916,0.005916,0.005916,0.004437,0.007394,0.007394,0.002958,0.005916,0.001479,0.005916,0.002958,0.005916,0.002958,0.004437,0.005916,0.005916,0.005916,0.004437,0.946488,0.319440,0.004437,0.004437,0.005916,0.005916,0.005916,0.005916,0.004437,0.005916,0.004437,0.002958,0.001479,0.001479,0.000000,0.001479,0.001479,0.0,0.000000,0.001479,0.000000,0.001479,0.000000,0.000000,0.000000,0.001479,0.000000,0.001479,0.001479,0.002958,0.002958,0.002958,0.026620,0.002958,0.017747,0.001479,0.002958,0.001479,0.001479,0.002958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.010336,0.025839,0.020671,0.015504,0.015504,0.020671,0.020671,0.020671,0.015504,0.020671,0.005168,0.020671,0.020671,0.015504,0.020671,0.020671,0.010336,0.025839,0.010336,0.020671,0.005168,0.010336,0.025839,0.020671,0.010336,0.020671,0.062014,0.532287,0.832022,0.010336,0.010336,0.020671,0.015504,0.015504,0.025839,0.015504,0.015504,0.015504,0.015504,0.

In [14]:
train_S['nerdiness']=train['nerdiness']

In [15]:
train_x = train_S.drop(['nerdiness'],axis=1)
train_y = train_S['nerdiness']

In [16]:
start_time=time.time()
lgb= GradientBoostingClassifier(
            n_estimators=1000
        )
lgb.fit(X=train_x,y=train_y)
end_time=time.time()
print('소요 시간 : ')
print(end_time-start_time)

소요 시간 : 
188.09440159797668


In [17]:
# 재학습 위한 공간

In [18]:
#lgbm_clf.fit(train_x, train_y)
lgbm_pred = lgb.predict(test)

In [19]:
submission = pd.read_csv('competition_data/sample_submission.csv')

submission

,index,nerdiness
0,0,-1
1,1,-1
2,2,-1
3,3,-1
4,4,-1
...,...,...
35447,35447,-1
35448,35448,-1
35449,35449,-1
35450,35450,-1


In [20]:
submission["nerdiness"] = lgbm_pred

In [21]:
submission

,index,nerdiness
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
35447,35447,1
35448,35448,0
35449,35449,1
35450,35450,0


In [22]:
submission.to_csv("baseline_notNA-iterative-withNorm-GBM.csv", index = False)